<h1>Build and train models</h1>

## Workshop overview

In this workshop, you will go through an end-to-end machine learning process. You will use the "AI4I 2020 Predictive Maintenance Dataset" from the UCI Machine Learning Repository https://archive.ics.uci.edu/ml/datasets/AI4I+2020+Predictive+Maintenance+Dataset, which contains information about machine failures, to train a binary classification model that predicts whether a machine will fail based on input data.

In this module, you will perform data exploration, data processing and model training in a familiar Jupyter Notebook environment using SageMaker Studio JupyterLab. In module 2 you will then deploy a SageMaker Inference Pipeline endpoint consisting of Feature Transformer and XGBoost models for real-time inference. Finally, in module 3, you will create a pipeline for the end-to-end process by using Amazon SageMaker Pipelines.

## In this notebook...

You will explore the data and use SKLearn Feature Transformers to preprocess the data. You then train the XGBoost model and test the models. You will use the SageMaker @remote decorator to scale data processing and model training as Amazon SageMaker Training jobs.

## Environment set up 

Let's get started by installing the requirements. A kernel restart might be required when the process completes.

In [ ]:
%pip install -r requirements.txt

In [ ]:
import os

# Set path to config file for the SageMaker Python SDK
os.environ["SAGEMAKER_USER_CONFIG_OVERRIDE"] = os.getcwd()

Thanks to the SageMaker Distribition the most popular ML frameworks and libraries are pre-intalled in the environment

In [ ]:
import pandas
import xgboost
import sklearn
import torch

print(f"Pandas version: {pandas.__version__}")
print(f"XGBoost version: {xgboost.__version__}")
print(f"SKLearn version: {sklearn.__version__}")

In [ ]:
%pip install seaborn

Download the dataset from the UCI website.

In [ ]:
import urllib
import os

input_data_dir = 'data/'
if not os.path.exists(input_data_dir):
    os.makedirs(input_data_dir)
input_data_path = os.path.join(input_data_dir, 'predictive_maintenance_raw_data_header.csv')
dataset_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00601/ai4i2020.csv"
urllib.request.urlretrieve(dataset_url, input_data_path)

# Explaratory Data Analysis

Here you will perform a fairly simple analysis to view the shape of the raw data, descriptive statistics of the features, frequency of the labels, and pairwise relationships between the features. Feel free to spend more time on EDA.

Find out how many samples and columns are included in the dataset.

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv(input_data_path)

print('The shape of the dataset is:', df.shape)

Preview the first 10 rows.

In [ ]:
df.head(10)

Check the data types for each column and identify columns with missing values.

In [ ]:
df.describe()

List the possible values for the "Machine failure" column and frequency of their occurence over the entire dataset.

In [ ]:
df['Machine failure'].value_counts()

Plot the target columns to visualise the distribution of values.

In [ ]:
import matplotlib.pyplot as plt

df['Machine failure'].value_counts().plot.bar()
plt.show()

We have discovered that the dataset is quite unbalanced. However, we are not going to balance it in this workshop.

In [ ]:
# compute the count of unique values for colums in df
df.nunique()

In [ ]:
# a function that removes a column from df
def remove_column(df, col):
    df = df.drop(col, axis=1)
    return df

Drop the attributes you are not interested in and keep only the numeric attributes.

In [ ]:
df1 = df.sample(frac =.1)
df1 = df1.drop(['UDI', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'], axis=1).select_dtypes(include='number')
df1.head()

In [ ]:
df1.info()

Use a pair plot to spot correlations.

In [ ]:
import seaborn
import matplotlib.pyplot as plt

seaborn.pairplot(df1, hue='Machine failure', corner=True)
plt.show()

To keep the data exploration step short during the workshop, no additional queries are included. However, feel free to explore the dataset more if you have time.

## Feature Engineering

### Data Processing

We execute data pre-processing with the following Python function; more specifically, we execute one-hot encoding of the relevant categorical columns and fill in the NaN values based on domain knowledge. Then, the code splits the dataset into training, validation, and test datasets, fits the featurizer model, and transforms the datasets. The function returns the model and the output datasets, and saves the serialized model to the file system.

The code below uses the Remote Decorator @remote to run the Python function as a SageMaker job, without having to perform any specific adaptation to the function code. For further information on the @remote decorator, please check: https://docs.aws.amazon.com/sagemaker/latest/dg/train-remote-decorator.html

We also use the SageMaker Training Managed Warm Pools feature by setting the keep_alive_period_in_seconds. Managed Warm Pools allow you to keep the training cluster alive for a defined period of time, so that the subsequent executions can reuse the same infrastructure and cached assets (e.g. container image) if possible. This helps significantly reduncing the initial latency of the jobs. For further information, please check: https://docs.aws.amazon.com/sagemaker/latest/dg/train-warm-pools.html

In [ ]:
import os
import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.exceptions import DataConversionWarning

from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from sagemaker.remote_function import remote

@remote(keep_alive_period_in_seconds=3600, job_name_prefix="amzn-sm-btd-preprocess")
def preprocess(df):
    columns = ['Type', 'Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]', 'Machine failure']
    cat_columns = ['Type']
    num_columns = ['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]']
    target_column = 'Machine failure'

    df = df[columns]

    training_ratio = 0.8
    validation_ratio = 0.1
    test_ratio = 0.1

    X = df.drop(target_column, axis=1)
    y = df[target_column]

    print(f'Splitting data training ({training_ratio}), validation ({validation_ratio}), and test ({test_ratio}) sets ')

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, random_state=0, stratify=y)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=validation_ratio/(validation_ratio+training_ratio), random_state=2, stratify=y_train)

    # Apply transformations
    transformer = ColumnTransformer(transformers=[('numeric', StandardScaler(), num_columns),
                                                  ('categorical', OneHotEncoder(), cat_columns)],
                                    remainder='passthrough')
    featurizer_model = transformer.fit(X_train)
    X_train = featurizer_model.transform(X_train)
    X_val = featurizer_model.transform(X_val)

    print(f'Shape of train features after preprocessing: {X_train.shape}')
    print(f'Shape of validation features after preprocessing: {X_val.shape}')
    print(f'Shape of test features after preprocessing: {X_test.shape}')
    
    y_train = y_train.values.reshape(-1)
    y_val = y_val.values.reshape(-1)
    
    print(f'Shape of train labels after preprocessing: {y_train.shape}')
    print(f'Shape of validation labels after preprocessing: {y_val.shape}')
    print(f'Shape of test labels after preprocessing: {y_test.shape}')

    model_file_path="/opt/ml/model/sklearn_model.joblib"
    os.makedirs(os.path.dirname(model_file_path), exist_ok=True)
    joblib.dump(featurizer_model, model_file_path)

    return X_train, y_train, X_val, y_val, X_test, y_test, featurizer_model

In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test, featurizer_model = preprocess(df)

Let's take a look at the featurizer model structure:

In [ ]:
featurizer_model

We can also show some rows from the processed training dataset.

In [ ]:
import pandas as pd
pd.DataFrame(X_train).head(10)

You will see that the categorical variables have been one-hot encoded, and you are free to check that we do not have NaN values anymore as expected.


## Model Training

In this part, you use XGboost to train a simple binary classification model using the pre-processed data generated in the previous step (preprocessing). Again, we are declaring a standard Python training function, that accepts some of the XGBoost hyperparameters as input ad returns the model.

In [ ]:
import os
import shutil
import xgboost
import numpy as np

from sagemaker.remote_function import remote

@remote(keep_alive_period_in_seconds=3600, job_name_prefix="amzn-sm-btd-train")
def train(X_train, y_train, X_val, y_val,
          eta=0.1, 
          max_depth=2, 
          gamma=0.0,
          min_child_weight=1,
          verbosity=0,
          objective='binary:logistic',
          eval_metric='auc',
          num_boost_round=5):

    print('Train features shape: {}'.format(X_train.shape))
    print('Train labels shape: {}'.format(y_train.shape))
    print('Validation features shape: {}'.format(X_val.shape))
    print('Validation labels shape: {}'.format(y_val.shape))

    # Creating DMatrix(es)
    dtrain = xgboost.DMatrix(X_train, label=y_train)
    dval = xgboost.DMatrix(X_val, label=y_val)
    watchlist = [(dtrain, "train"), (dval, "validation")]

    print('')
    print (f'===Starting training with max_depth {max_depth}===')

    param_dist = {
        "max_depth": max_depth,
        "eta": eta,
        "gamma": gamma,
        "min_child_weight": min_child_weight,
        "verbosity": verbosity,
        "objective": objective,
        "eval_metric": eval_metric
    }

    xgb = xgboost.train(
        params=param_dist,
        dtrain=dtrain,
        evals=watchlist,
        num_boost_round=num_boost_round)

    predictions = xgb.predict(dval)

    print ("Metrics for validation set")
    print('')
    print (pd.crosstab(index=y_val, columns=np.round(predictions),
                       rownames=['Actuals'], colnames=['Predictions'], margins=True))
    print('')

    rounded_predict = np.round(predictions)

    val_accuracy = accuracy_score(y_val, rounded_predict)
    val_precision = precision_score(y_val, rounded_predict)
    val_recall = recall_score(y_val, rounded_predict)

    print("Accuracy Model A: %.2f%%" % (val_accuracy * 100.0))
    print("Precision Model A: %.2f" % (val_precision))
    print("Recall Model A: %.2f" % (val_recall))

    from sklearn.metrics import roc_auc_score

    val_auc = roc_auc_score(y_val, predictions)
    print("Validation AUC A: %.2f" % (val_auc))

    model_file_path="/opt/ml/model/xgboost_model.bin"
    os.makedirs(os.path.dirname(model_file_path), exist_ok=True)
    xgb.save_model(model_file_path)

    return xgb

In [ ]:
eta=0.3
max_depth=8

booster = train(X_train, y_train, X_val, y_val,
              eta=eta, 
              max_depth=max_depth)

In [ ]:
booster

### Using the models to generate predictions

Finally you use the models for inference and evaluate model accuracy.

In [ ]:
#@remote(keep_alive_period_in_seconds=600, job_name_prefix="amzn-sm-btd-test")
def test(featurizer_model, booster, X_test, y_test):

    X_test = featurizer_model.transform(X_test)
    y_test = y_test.values.reshape(-1)

    dtest = xgboost.DMatrix(X_test, label=y_test)
    test_predictions = booster.predict(dtest)
    
    print ("===Metrics for Test Set===")
    print('')
    print (pd.crosstab(index=y_test, columns=np.round(test_predictions), 
                                     rownames=['Actuals'], 
                                     colnames=['Predictions'], 
                                     margins=True)
          )
    print('')

    rounded_predict = np.round(test_predictions)

    accuracy = accuracy_score(y_test, rounded_predict)
    precision = precision_score(y_test, rounded_predict)
    recall = recall_score(y_test, rounded_predict)
    print('')

    print("Accuracy Model A: %.2f%%" % (accuracy * 100.0))
    print("Precision Model A: %.2f" % (precision))
    print("Recall Model A: %.2f" % (recall))

    from sklearn.metrics import roc_auc_score

    auc = roc_auc_score(y_test, test_predictions)
    print("AUC A: %.2f" % (auc))

In [ ]:
test(featurizer_model, booster, X_test, y_test)